# Update Commissioners 2022

After redistricting, all commissioners are in new districts (all districts have new IDs), so they all need a new row in the commissioners table. 

A medium-term project will be upgrading the commissioners table so there is one row per commissioner-term.

In [1]:
import os
os.chdir('../')

import pytz
import pandas as pd
from datetime import datetime

In [2]:
people = pd.read_csv('data/people.csv')
results = pd.read_csv('data/dcboe/candidate_votes.csv')
write_in_winners = pd.read_csv('data/dcboe/write_in_winners.csv')
lookup = pd.read_csv('data/external_id_lookup.csv')

In [3]:
results = results[results.election_year == 2022].copy()
write_in_winners = write_in_winners[write_in_winners.election_year == 2022].copy()

In [4]:
winners = pd.concat([
    results[(results.winner) & (~results.write_in_winner)][['external_id', 'smd_id']]
    , write_in_winners[['external_id', 'smd_id']]
]).sort_values(by='smd_id')

winners

,external_id,smd_id
698,5562e6cb282ae08f0a57700a9f21b2ac6b13b6082f4088...,smd_2022_1A01
700,dcca894eb6fbad029f0602aea05dba22af14b0866045b7...,smd_2022_1A02
702,f9e78f92331964dbb13882c00022c44b62a83a73673a07...,smd_2022_1A03
704,1088a5b7560f9a6cdccb70b673efd8f2422cb8a8311aaf...,smd_2022_1A04
706,d84a47f58fa9fd4e3d347fadbfd177f3a344945eca2c04...,smd_2022_1A05
...,...,...
1408,c141ce896c3be224355903d852999f39797e44706b9634...,smd_2022_8F01
1410,286ebfc5cc3bff8ffb8d3dc70ec5208edfe925f9bb1a75...,smd_2022_8F02
1412,934770ff4ec88d5f0834addb6f8f00894bfeeb1a1b8d62...,smd_2022_8F03
1414,03e06e6cea8a0a69c48caf8221946ddbb8b13a6ca8fe28...,smd_2022_8F04


In [5]:
winners_lookup = pd.merge(winners, lookup, how='inner', on='external_id').drop('full_name', axis=1)
winners_lookup_people = pd.merge(winners_lookup, people, how='inner', on='person_id')

In [6]:
# Confirm that there is only one winner per SMD
winners_lookup_people.groupby('smd_id').size().max() == 1

True

In [7]:
# The dates go into the google sheet as text, they don't need to be datetime
winners_lookup_people['start_date'] = '2023-01-02'
winners_lookup_people['end_date'] = '2025-01-02'

In [8]:
columns_to_clipboard = ['person_id', 'smd_id', 'full_name', 'start_date', 'end_date']
winners_lookup_people[columns_to_clipboard]

,person_id,smd_id,full_name,start_date,end_date
0,10460,smd_2022_1A01,Max Ewart,2023-01-02,2025-01-02
1,10285,smd_2022_1A02,Dieter Lehmann Morales,2023-01-02,2025-01-02
2,11345,smd_2022_1A03,Carlo Perri,2023-01-02,2025-01-02
3,11371,smd_2022_1A04,Jeremy Sherman,2023-01-02,2025-01-02
4,11417,smd_2022_1A05,Stephen Coleman Kenny,2023-01-02,2025-01-02
...,...,...,...,...,...
321,11542,smd_2022_8F01,Nic Wilson,2023-01-02,2025-01-02
322,11543,smd_2022_8F02,Rick Murphree,2023-01-02,2025-01-02
323,11406,smd_2022_8F03,Brian Strege,2023-01-02,2025-01-02
324,10210,smd_2022_8F04,Edward Daniels,2023-01-02,2025-01-02


In [9]:
# Copy to "commissioners" tab
winners_lookup_people[columns_to_clipboard].to_clipboard(index=False)